In [1]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import json as js
import ast as ast
import re

In [2]:
#Convertir el archivo 'output_steam_games.json' en dataframe

DF_Juegos_steam = pd.read_json('output_steam_games.json', lines=True)

In [3]:
def transformar_listas(df, columna):
    """
    Transforma listas a cadenas, dejando nulos como están
    
    Returns:
    pd.DataFrame: El DataFrame con la columna transformada.
    """
    df[columna] = df[columna].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [4]:
transformar_listas(DF_Juegos_steam,'publisher')
transformar_listas(DF_Juegos_steam,'genres')
transformar_listas(DF_Juegos_steam,'app_name')
transformar_listas(DF_Juegos_steam,'title')
transformar_listas(DF_Juegos_steam,'url')
transformar_listas(DF_Juegos_steam,'release_date')
transformar_listas(DF_Juegos_steam,'reviews_url')
transformar_listas(DF_Juegos_steam,'specs')
transformar_listas(DF_Juegos_steam,'price')
transformar_listas(DF_Juegos_steam,'early_access')
transformar_listas(DF_Juegos_steam,'id')
transformar_listas(DF_Juegos_steam,'developer')


In [5]:
# Reemplazar nulos en app_name con valores de title
DF_Juegos_steam['app_name'] = DF_Juegos_steam['app_name'].fillna(DF_Juegos_steam['title'])

# Reemplazar nulos en title con valores de app_name
DF_Juegos_steam['title'] = DF_Juegos_steam['title'].fillna(DF_Juegos_steam['app_name'])

# Reemplazar nulos en tags con valores de genre
DF_Juegos_steam['tags'] = DF_Juegos_steam['tags'].fillna(DF_Juegos_steam['genres'])

# Reemplazar nulos en genre con valores de tags
DF_Juegos_steam['genres'] = DF_Juegos_steam['genres'].fillna(DF_Juegos_steam['tags'])

In [6]:
#Eliminamos columnas que no consideremos necesarias

DF_Juegos_steam = DF_Juegos_steam.drop(columns='title')
#DF_Juegos_steam = DF_Juegos_steam.drop(columns='early_access')
DF_Juegos_steam = DF_Juegos_steam.drop(columns='tags')

In [7]:
#Eliminamos las filas en donde todos los valores sean nulos
DF_Juegos_steam = DF_Juegos_steam.dropna(how='all')
print(DF_Juegos_steam)

               publisher                                             genres  \
88310          Kotoshiro        Action, Casual, Indie, Simulation, Strategy   
88311   Making Fun, Inc.                 Free to Play, Indie, RPG, Strategy   
88312       Poolians.com    Casual, Free to Play, Indie, Simulation, Sports   
88313               彼岸领域                          Action, Adventure, Casual   
88314               None                    [Action, Indie, Casual, Sports]   
...                  ...                                                ...   
120440   Ghost_RUS Games                Casual, Indie, Simulation, Strategy   
120441            Sacada                            Casual, Indie, Strategy   
120442      Laush Studio                          Indie, Racing, Simulation   
120443          SIXNAILS                                      Casual, Indie   
120444              None  [Early Access, Adventure, Indie, Action, Simul...   

                        app_name  \
88310        Lo

In [8]:
# Filtrar el DataFrame donde app_name es nulo
null_app_name_df = DF_Juegos_steam[DF_Juegos_steam['app_name'].isnull()]

print("Filas donde app_name es nulo:")
print(null_app_name_df)

Filas donde app_name es nulo:
      publisher         genres app_name  \
88384      None           None     None   
90890      None  Action, Indie     None   

                                               url release_date  \
88384               http://store.steampowered.com/         None   
90890  http://store.steampowered.com/app/317160/_/   2014-08-26   

                                             reviews_url  \
88384                                               None   
90890  http://steamcommunity.com/app/317160/reviews/?...   

                          specs  price  early_access        id developer  
88384                      None  19.99           0.0       NaN      None  
90890  Single-player, Game demo   None           0.0  317160.0      None  


In [9]:
#Buscamos la fila 88384
fila_88384 = DF_Juegos_steam.loc[88384]

print("Fila con índice 88384:")
print(fila_88384)



Fila con índice 88384:
publisher                                 None
genres                                    None
app_name                                  None
url             http://store.steampowered.com/
release_date                              None
reviews_url                               None
specs                                     None
price                                    19.99
early_access                               0.0
id                                         NaN
developer                                 None
Name: 88384, dtype: object


In [10]:
# Eliminar la fila con índice 88384
DF_Juegos_steam = DF_Juegos_steam.drop(index=88384)

In [11]:
# Inicializar un diccionario para contar ocurrencias
contador = {}

# Contar ocurrencias de cada valor en la columna 'id'
for valor in DF_Juegos_steam['id']:
    if valor in contador:
        contador[valor] += 1
    else:
        contador[valor] = 1

# Obtener los IDs duplicados
ids_duplicados = [k for k, v in contador.items() if v > 1]

if ids_duplicados:
    print('IDs de los valores duplicados en la columna id:')
    print(ids_duplicados)
else:
    print('No hay valores duplicados en la columna id')

IDs de los valores duplicados en la columna id:
[612880.0]


In [12]:
# Filtrar la fila donde el id sea 612880
fila = DF_Juegos_steam[DF_Juegos_steam['id'] == 612880]

print("Fila donde el id es 612880:")
print(fila)

Fila donde el id es 612880:
                 publisher  genres                          app_name  \
102204  Bethesda Softworks  Action  Wolfenstein II: The New Colossus   
102883  Bethesda Softworks  Action  Wolfenstein II: The New Colossus   

                                                      url release_date  \
102204          http://store.steampowered.com/app/612880/   2017-10-26   
102883  http://store.steampowered.com/app/612880/Wolfe...   2017-10-26   

                                              reviews_url  \
102204  http://steamcommunity.com/app/612880/reviews/?...   
102883  http://steamcommunity.com/app/612880/reviews/?...   

                                                    specs  price  \
102204  Single-player, Steam Achievements, Full contro...  59.99   
102883  Single-player, Steam Achievements, Full contro...  59.99   

        early_access        id      developer  
102204           0.0  612880.0  Machine Games  
102883           0.0  612880.0  Machine Games  

In [13]:
# Eliminar una sola fila duplicada (manteniendo la primera)
DF_Juegos_steam = DF_Juegos_steam.drop_duplicates(subset='id', keep='first')
print(DF_Juegos_steam)

               publisher                                             genres  \
88310          Kotoshiro        Action, Casual, Indie, Simulation, Strategy   
88311   Making Fun, Inc.                 Free to Play, Indie, RPG, Strategy   
88312       Poolians.com    Casual, Free to Play, Indie, Simulation, Sports   
88313               彼岸领域                          Action, Adventure, Casual   
88314               None                    [Action, Indie, Casual, Sports]   
...                  ...                                                ...   
120440   Ghost_RUS Games                Casual, Indie, Simulation, Strategy   
120441            Sacada                            Casual, Indie, Strategy   
120442      Laush Studio                          Indie, Racing, Simulation   
120443          SIXNAILS                                      Casual, Indie   
120444              None  [Early Access, Adventure, Indie, Action, Simul...   

                        app_name  \
88310        Lo

In [14]:
#Transforma listas a cadenas, dejando nulos como están
DF_Juegos_steam['genres'] = DF_Juegos_steam['genres'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)


In [ ]:
DF_Juegos_steam.reset_index(inplace=True)

In [16]:
#Guardamos el DataFrame con el ETL realizado
DF_Juegos_steam.to_json('DF_Juegos_steam.json', orient='records', lines=True)